In [92]:
import argparse
import polars as pl

bakta = '/mmfs1/gscratch/pedslabs_hoffman/carsonjm/CFPhageome/repos/UHVDB/toolkit/work/ea/f4025c0345022cbcb25cd268aaa62f/samplesheet_se_fastq_gcode11.part_001.fna.tsv.gz'
foldseek = '/mmfs1/gscratch/pedslabs_hoffman/carsonjm/CFPhageome/repos/UHVDB/toolkit/work/ea/f4025c0345022cbcb25cd268aaa62f/samplesheet_se_fastq_gcode11.part_001.fna_nohit_v_refDB.m8.gz'
pharokka = '/mmfs1/gscratch/pedslabs_hoffman/carsonjm/CFPhageome/repos/UHVDB/toolkit/work/ea/f4025c0345022cbcb25cd268aaa62f/samplesheet_se_fastq_gcode11.part_001.fna.pharokka.tsv.gz'
phold = '/mmfs1/gscratch/pedslabs_hoffman/carsonjm/CFPhageome/repos/UHVDB/toolkit/work/ea/f4025c0345022cbcb25cd268aaa62f/samplesheet_se_fastq_gcode11.part_001.fna.phold.tsv.gz'
phynteny = '/mmfs1/gscratch/pedslabs_hoffman/carsonjm/CFPhageome/repos/UHVDB/toolkit/work/ea/f4025c0345022cbcb25cd268aaa62f/samplesheet_se_fastq_gcode11.part_001.fna.phynteny.tsv.gz'

# load Bakta TSV
bakta = (
    pl.read_csv(bakta, separator='\t', skip_rows=5)
        .rename({'#Sequence Id': 'Sequnce Id'})
)

# load foldseek TSV
try:
    foldseek = (
        pl.read_csv(foldseek, separator='\t', has_header=False,
            new_columns=["Locus Tag" , "Foldseek Reference", "Foldseek Identity", "Foldseek Align Length", "mismatch", "gapopen", "qstart", "qend", "tstart", "tend", "Foldseek E-value", "Foldseek Bitscore"]
        )
        .drop(['mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend'])
    )
    bakta_foldseek = bakta.join(foldseek, on='Locus Tag', how='left')
except pl.exceptions.NoDataError:
    bakta_foldseek = bakta.with_columns([
        pl.lit(None).alias('Foldseek Reference'),
        pl.lit(None).alias('Foldseek Identity'),
        pl.lit(None).alias('Foldseek Align Length'),
        pl.lit(None).alias('Foldseek E-value'),
        pl.lit(None).alias('Foldseek Bitscore'),
    ])

In [93]:
bakta_foldseek

Sequnce Id,Type,Start,Stop,Strand,Locus Tag,Gene,Product,DbXrefs,Foldseek Reference,Foldseek Identity,Foldseek Align Length,Foldseek E-value,Foldseek Bitscore
str,str,i64,i64,str,str,str,str,str,null,null,null,null,null
"""MT192765.1""","""cds""",259,13476,"""+""","""MT192765.1_01""","""r1A_SARS2""","""Replicase polyprotein 1a""","""UniRef:UniRef50_P0DTC1, UserPr…",null,null,null,null,null
"""MT192765.1""","""cds""",13803,21548,"""+""","""MT192765.1_02""",null,"""hypothetical protein""",null,null,null,null,null,null
"""MT192765.1""","""cds""",21556,25377,"""+""","""MT192765.1_03""","""sPIKE_SARS2""","""Spike glycoprotein""","""UniRef:UniRef50_P0DTC2, UserPr…",null,null,null,null,null
"""MT192765.1""","""cds""",25386,26213,"""+""","""MT192765.1_04""","""aP3A_SARS2""","""ORF3a protein""","""UniRef:UniRef50_P0DTC3, UserPr…",null,null,null,null,null
"""MT192765.1""","""cds""",26516,27184,"""+""","""MT192765.1_05""","""vME1_SARS2""","""Membrane protein""","""UniRef:UniRef50_P0DTC5, UserPr…",null,null,null,null,null
"""MT192765.1""","""cds""",27387,27752,"""+""","""MT192765.1_06""","""nS7A_SARS2""","""ORF7a protein""","""UniRef:UniRef50_P0DTC7, UserPr…",null,null,null,null,null
"""MT192765.1""","""cds""",27887,28252,"""+""","""MT192765.1_07""","""nS8_SARS2""","""ORF8 protein""","""UniRef:UniRef50_P0DTC8, UserPr…",null,null,null,null,null
"""MT192765.1""","""cds""",28267,29526,"""+""","""MT192765.1_08""","""nCAP_SARS2""","""Nucleoprotein""","""UniRef:UniRef50_P0DTC9, UserPr…",null,null,null,null,null
"""MT192765.1""","""ncRNA-region""",29596,29655,"""+""",null,null,"""Coronavirus 3' UTR pseudoknot""","""RFAM:RF00165, SO:0005836""",null,null,null,null,null


In [94]:
# load pharokka TSV
pharokka = (
    pl.read_csv(pharokka, separator='\t', 
        columns=['gene', 'pyhmmer_bitscore', 'pyhmmer_evalue', 'phrog', 'annot', 'category']
    )
    .rename({
        'gene': 'Locus Tag', 'phrog': 'Pharokka Phrog', 'annot': 'Pharokka Annotation',
        'pyhmmer_bitscore':'Pharokka Bitscore', 'pyhmmer_evalue':'Pharokka E-value',
        'category': 'Pharokka Category'
    })
)

# load phold TSV
phold = (
    pl.read_csv(phold, separator='\t', 
        columns=['cds_id', 'phrog', 'function', 'product', 'annotation_method', 'annotation_confidence', 'bitscore', 'fident', 'evalue', 'prostt5_confidence']
    )
    .filter(~pl.col('annotation_method').is_in(['pharokka', 'none']))
    .rename({
        'cds_id':'Locus Tag', 'phrog':'Phold Phrog', 'function':'Phold Category',
        'product':'Phold Annotation', 'annotation_method':'Annotation Method',
        'annotation_confidence':'Annotation Confidence', 'bitscore':'Phold bitscore',
        'fident':'Phold fident', 'evalue':'Phold evalue', 'prostt5_confidence':'ProstT5 Confidence'
    })
    .drop(['Annotation Method'])
)

# load phynteny TSV
phynteny = (
    pl.read_csv(phynteny, separator='\t', 
        columns=['ID', 'phrog_id', 'phynteny_category', 'phynteny_confidence']
    )
    .rename({
        'ID':'Locus Tag', 'phrog_id':'Phynteny Phrog',
        'phynteny_category':'Phynteny Category', 'phynteny_confidence':'Phynteny Confidence'
    })
)

# combine phrog annotations
phrogs = pharokka.join(phold, on='Locus Tag', how='left').join(phynteny, on=['Locus Tag'], how='left')

In [95]:
phrogs

Locus Tag,Pharokka Bitscore,Pharokka E-value,Pharokka Phrog,Pharokka Annotation,Pharokka Category,Phold Phrog,Phold Category,Phold Annotation,Annotation Confidence,Phold bitscore,Phold fident,Phold evalue,ProstT5 Confidence,Phynteny Phrog,Phynteny Category,Phynteny Confidence
str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,str,str,str
"""MT192765.1_CDS_0001""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""hypothetical protein""","""unknown function""",null,null,null,null,null,null,null,null,null,null,null
"""MT192765.1_CDS_0002""","""45.270847""","""9.97060101097363e-12""","""25411""","""hypothetical protein""","""unknown function""","""25411""","""DNA, RNA and nucleotide metabo…","""helicase""","""low""",670.0,0.172,1.0100e-14,29.953125,null,null,null
"""MT192765.1_CDS_0003""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""hypothetical protein""","""unknown function""",null,null,null,null,null,null,null,null,null,null,null
"""MT192765.1_CDS_0004""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""hypothetical protein""","""unknown function""",null,null,null,null,null,null,null,null,null,null,null
"""MT192765.1_CDS_0005""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""hypothetical protein""","""unknown function""",null,null,null,null,null,null,null,null,null,null,null
"""MT192765.1_CDS_0006""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""hypothetical protein""","""unknown function""",null,null,null,null,null,null,null,null,null,null,null
"""MT192765.1_CDS_0007""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""hypothetical protein""","""unknown function""",null,null,null,null,null,null,null,null,null,null,null
"""MT192765.1_CDS_0008""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""No_PHROGs_HMM""","""hypothetical protein""","""unknown function""",null,null,null,null,null,null,null,null,null,null,null


In [ ]:
padloc = '/mmfs1/gscratch/pedslabs_hoffman/carsonjm/CFPhageome/repos/UHVDB/toolkit/work/63/e7fe3255a2e9c1ff37e7f4b7520862/samplesheet_defense_gcode11.part_001.fna.padloc.csv.gz'

# load padloc CSV
padloc = (
    pl.read_csv(padloc,
        columns=[
            'system.number', 'seqid', 'system', 'target.name', 'hmm.accession', 'hmm.name',
            'protein.name', 'full.seq.E.value', 'domain.iE.value', 'target.coverage', 'hmm.coverage',
            'start', 'end', 'strand', 'target.description', 'relative.position', 'contig.end', 'all.domains',
            'best.hits'
        ]
    )
    .drop([
        'seqid', 'hmm.accession', 'hmm.name', 'start', 'end', 'target.description', 'strand',
        'relative.position', 'all.domains', 'best.hits', 'contig.end'
    ])
    .rename({
        'system.number': 'PADLOC System Number', 'system':'PADLOC System', 'target.name':'Locus Tag',
        'full.seq.E.value':'PADLOC Full E-value', 'domain.iE.value':'PADLOC Domain E-value',
        'target.coverage':'PADLOC Query Coverage', 'hmm.coverage':'PADLOC HMM Coverage',
    })
)

padloc.head(3)

PADLOC System Number,PADLOC System,Locus Tag,protein.name,PADLOC Full E-value,PADLOC Domain E-value,PADLOC Query Coverage,PADLOC HMM Coverage
i64,str,str,str,f64,f64,f64,f64
1,"""RosmerTA""","""NZ_QOYD01000051.1_014""","""RmrT""",3.2000e-94,3.6000e-94,0.995,0.99
1,"""RosmerTA""","""NZ_QOYD01000051.1_015""","""RmrA""",2.3000e-202,2.6000e-202,0.984,0.992


In [79]:
defensefinder = '/mmfs1/gscratch/pedslabs_hoffman/carsonjm/CFPhageome/repos/UHVDB/toolkit/work/3f/e3dae7336ba138cc521c2a16bcecc6/samplesheet_defense_gcode11.part_001.fna.defense_finder_genes.tsv.gz'

# load defensefinder TSV
defensefinder = (
    pl.read_csv(defensefinder, separator='\t',
        columns=[
            'hit_id', 'gene_name', 'sys_wholeness', 'sys_score', 'hit_status', 'hit_i_eval', 'hit_score', 'type', 'subtype', 'activity'
        ]
    )
    .rename({
        'hit_id': 'Locus Tag', 'gene_name':'DF Reference', 'sys_wholeness':'DF System Wholeness', 'hit_status':'DF Hit Status',
        'hit_i_eval':'DF E-value', 'hit_score':'DF Bitscore', 'type':'DF Type',
        'subtype':'DF Subtype', 'activity':'DF Activity'
    })
)

defensefinder.head(3)

Locus Tag,DF Reference,DF System Wholeness,sys_score,DF Hit Status,DF E-value,DF Bitscore,DF Type,DF Subtype,DF Activity
str,str,f64,f64,str,f64,f64,str,str,str
"""NZ_QOYD01000051.1_014""","""RosmerTA__RmrT_2585209417""",1.0,2.0,"""mandatory""",3.6000e-94,306.8,"""RosmerTA""","""RosmerTA""","""Defense"""
"""NZ_QOYD01000051.1_015""","""RosmerTA__RmrA_2585209417""",1.0,2.0,"""mandatory""",2.6000e-202,664.5,"""RosmerTA""","""RosmerTA""","""Defense"""
